# Credit EDA With Python

Let's explore credit data present in this [link](https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/develop/dataset/credito.csv). The data is in CSV format and contains information about a financial institution's customers.

# 1. Problem Overview

Our primary focus lies on elucidating the intricacies of the second column, denoted as default. This column serves as a binary indicator, distinguishing customers who are not in default (`default = 0`) from those in default (`default = 1`). 

Our aim is to discern the factors contributing to a client's failure to meet financial obligations, drawing insights from attributes such as salary, education, and financial transactions.

> In this context, the attribute of interest, namely default, assumes the role of the **response variable** or **dependent variable**, while attributes like (`age`, `salary`, etc.), function as **explanatory variables**, independent variables, or even predictor variables.

| Coluna  | Descrição |
| ------- | --------- |
| id      | Número da conta |
| default | Indica se o cliente é adimplente (0) ou inadimplente (1) |
| idade   | --- |
| sexo    | --- |
| depedentes | --- |
| escolaridade | --- |
| estado_civil | --- |
| salario_anual | Faixa do salario mensal multiplicado por 12 |
| tipo_cartao | Categoria do cartao: blue, silver, gold e platinium |
| meses_de_relacionamento | Quantidade de meses desde a abertura da conta |
| qtd_produtos | Quantidade de produtos contratados |
| iteracoes_12m | Quantidade de iteracoes com o cliente no último ano |
| meses_inatico_12m | Quantidade de meses que o cliente ficou inativo no último ano |
| limite_credito | Valor do limite do cartão de crédito |
| valor_transacoes_12m | Soma total do valor das transações no cartão de crédito no último ano |
| qtd_transacoes_12m | Quantidade total de transações no cartão de crédito no último ano |

# 2. Importing Libraries

In this step, we ensure we have the essential tools by importing the necessary libraries. This includes leveraging popular libraries like Pandas, NumPy, and Matplotlib, which empower us to efficiently manipulate data and create insightful visualizations. This foundational setup establishes a robust framework for our data exploration and analysis.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import sklearn

# 3. Dataset - Loading/Viewing/Generation

Our journey begins with the foundational step of loading data into a `Pandas` dataframe:

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/develop/dataset/credito.csv', na_values='na')

In [ ]:
df.head(n=10)

# 4. Exploration stage

Now that we have our dataset loaded, it's time to delve into its structure and relationships. We'll employ a correlation matrix, visually represented by a heatmap.

This exploration step lays the groundwork for uncovering patterns and gaining valuable insights from our data.

In [ ]:
# Filter numeric columns for correlation analysis
numeric_columns = df.select_dtypes(include=['number']).columns
corr_matrix = df[numeric_columns].corr()

# Identify highly correlated variables (correlation > 0.75)
high_corr_var = []
for col in corr_matrix:
    highly_corr_rows = corr_matrix[col][corr_matrix[col] > 0.75].index
    high_corr_var.extend([[col, row, corr_matrix[col][row]] for row in highly_corr_rows if row != col])

# Display the highly correlated variables
print(high_corr_var)

# Drop one of the redundant correlated variables
if high_corr_var:
    df.drop(high_corr_var[0][1], axis=1, inplace=True)

# Visualize the correlation matrix using a heatmap
plt.figure(figsize=(10, 8))
heatmap = sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)

# Highlight the highly correlated variables on the heatmap
for var in high_corr_var:
    correlation_value = corr_matrix.loc[var[0], var[1]]
    heatmap.text(
        numeric_columns.get_loc(var[1]) + 0.5,  # Adjusted the column index
        numeric_columns.get_loc(var[0]) + 0.5,  # Adjusted the row index
        f"{var[0]} & {var[1]} ({correlation_value:.2f})",
        color='red',
        ha='center',
        va='center',
        fontsize=10,
        rotation=45  # Rotated the text for better visibility
    )

# Display the title and heatmap
plt.title('Correlation Matrix')

# Remove red text from the graph
plt.text(0, 0, '', color='red')

plt.show()

In [ ]:
df.shape # retorna uma tupla (qtd linhas, qtd colunas)

In [ ]:
df[df['default'] == 0].shape

In [ ]:
df[df['default'] == 1].shape

In [ ]:
qtd_total, _ = df.shape
qtd_adimplentes, _ = df[df['default'] == 0].shape
qtd_inadimplentes, _ = df[df['default'] == 1].shape

In [ ]:
print(f"The proportion of paying customers is {round(100 * qtd_adimplentes / qtd_total, 2)}%")
print(f"The proportion of defaulting customers is {round(100 * qtd_inadimplentes / qtd_total, 2)}%")

# 5. Cleaning and Transformation

First lets see the data types and its attributes So then we can make the necessary adjustments regarding data standardization and cleaning

In [ ]:
df.head(n=5)

* **Columns** and their respective **data types**.

In [ ]:
# Display data types of each column
print(df.dtypes)

- **Categorical** attributes.

In [ ]:
df.select_dtypes('object').describe().transpose()

- **Numeric** attributes.

In [ ]:
df.drop('id', axis=1).select_dtypes('number').describe().transpose()

Missing data could be:

  - Empty (`""`);
  - Nulls (`None`);
  - Not available or applicable (`na`, `NA`, etc.);
  - Non-numeric (`nan`, `NaN`, `NAN`, etc).

In [ ]:
df.head()

We can check which columns have missing data.

In [ ]:
df.isna().any()

- The function below generates some statistics on the missing data columns.

In [ ]:
def stats_dados_faltantes(df: pd.DataFrame) -> None:

  stats_dados_faltantes = []
  for col in df.columns:
    if df[col].isna().any():
      qtd, _ = df[df[col].isna()].shape
      total, _ = df.shape
      dict_dados_faltantes = {col: {'quantidade': qtd, "porcentagem": round(100 * qtd/total, 2)}}
      stats_dados_faltantes.append(dict_dados_faltantes)

  for stat in stats_dados_faltantes:
    print(stat)

In [ ]:
stats_dados_faltantes(df=df)

In [ ]:
stats_dados_faltantes(df=df[df['default'] == 0])

In [ ]:
stats_dados_faltantes(df=df[df['default'] == 1])

**# Data Wrangling**

Now that we have gained a deeper understanding of our dataset, it's time to perform data wrangling, a crucial step in preparing the data for analysis. Our focus will be on:

  - Correcting the column schema for better interpretation;
  - Handling missing data effectively.

During the exploration stage, we observed that the columns 'limite_credito' and 'valor_transacoes_12m' were incorrectly interpreted as categorical columns (`dtype = object`).

Let's proceed with the necessary transformations.

In [ ]:
df[['limite_credito', 'valor_transacoes_12m']].dtypes

In [ ]:
df[['limite_credito', 'valor_transacoes_12m']].head(n=5)

Now, we'll implement a `lambda` function for data cleaning. Before applying this function, let's conduct a preliminary test using the `map` functional method to ensure its effectiveness:

In [ ]:
fn = lambda valor: float(valor.replace(".", "").replace(",", "."))

valores_originais = ['12.691,51', '8.256,96', '3.418,56', '3.313,03', '4.716,22']
valores_limpos = list(map(fn, valores_originais))

print(valores_originais)
print(valores_limpos)

Now that our `lambda` cleaning function is prepared, effortlessly apply it to the selected columns for a seamless data transformation.

In [ ]:
df['valor_transacoes_12m'] = df['valor_transacoes_12m'].apply(lambda x: fn(x) if isinstance(x, str) else x)
df['limite_credito'] = df['limite_credito'].apply(lambda x: fn(x) if isinstance(x, str) else x)

Let's revisit the description of the dataset structure:

In [ ]:
df.dtypes

- **Categorical** attributes.

In [ ]:
df.select_dtypes('object').describe().transpose()

- **Numeric** attributes.

In [ ]:
df.drop('id', axis=1).select_dtypes('number').describe().transpose()

Since pandas is aware of missing data, removing the problematic rows is trivial.

In [ ]:
df.dropna(inplace=True)

Let's look at the data structure again.

In [ ]:
df.shape

In [ ]:
df[df['default'] == 0].shape

In [ ]:
df[df['default'] == 1].shape

In [ ]:
qtd_total_novo, _ = df.shape
qtd_adimplentes_novo, _ = df[df['default'] == 0].shape
qtd_inadimplentes_novo, _ = df[df['default'] == 1].shape

In [ ]:
print(f"The proportion of active defaulters is {round(100 * qtd_adimplentes / qtd_total, 2)}%")
print(f"The new proportion of paying customers is {round(100 * qtd_adimplentes_novo / qtd_total_novo, 2)}%")
print(f"The proportion of defaulting customers is {round(100 * qtd_inadimplentes / qtd_total, 2)}%")
print(f"The new proportion of defaulting customers is {round(100 * qtd_inadimplentes_novo / qtd_total_novo, 2)}%")

# 6. Exploratory Analysis with Visualizations

Now that the data is prepared, our next step involves creating various visualizations to uncover correlations between explanatory variables and the response variable. This will help us discern the factors that might contribute to a customer defaulting. Our approach involves comparing the entire customer base with subsets of compliant and non-compliant customers.

To kick off this analysis, we import the necessary visualization libraries and segment the dataset into compliant and non-compliant customer groups.

In [ ]:
sns.set_style("whitegrid")

In [ ]:
df_adimplente = df[df['default'] == 0]

In [ ]:
df_inadimplente = df[df['default'] == 1]

### Categorical Views

*In this* section, we'll explore the relationship between the **default** response variable and categorical attributes through insightful visualizations.

In [ ]:
df.select_dtypes('object').head(n=5)

* "Escolaridade"

In [ ]:
coluna = 'escolaridade'
titulos = ['Escolaridade dos Clientes', 'Escolaridade dos Clientes Adimplentes', 'Escolaridade dos Clientes Inadimplentes']

# Manually set colors for each category
colors = {'sem educacao formal': 'blue', 'ensino medio': 'orange', 'graduacao': 'green', 'mestrado': 'red', 'doutorado': 'purple'}

# Sort unique values alphabetically
categories = sorted(df[coluna].unique())

# Create a figure with three subplots
figura, eixos = plt.subplots(1, 3, figsize=(20, 5))

# Use a loop to make a bar plot for each dataframe
for i in range(3):
    # Choose the dataframe and title according to the index
    if i == 0:
        dataframe = df
        titulo = titulos[0]
    elif i == 1:
        dataframe = df_adimplente
        titulo = titulos[1]
    else:
        dataframe = df_inadimplente
        titulo = titulos[2]

    # Count the values in the column and make a bar plot
    f = sns.countplot(x=coluna, data=dataframe, ax=eixos[i], palette=[colors[cat] for cat in categories])
    f.set(title=titulo, xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')
    f.set_xticklabels(labels=categories, rotation=90)

    # Set standard y-axis scale
    f.set(ylim=(0, max(df[coluna].value_counts()) + 5))

# Adjust the spacing between subplots
figura.tight_layout()

figura.show()

 - Salário Anual

In [ ]:
# Manually set colors for each category
colors = {'120k+': 'purple', '40k - 60k': 'orange', '60k - 80k': 'green', '80k - 120k': 'red', 'menos que 40k': 'blue'}

# Create a figure with three subplots
figura, eixos = plt.subplots(1, 3, figsize=(20, 5), sharex=True)

max_y = 0

for dataframe, titulo, eixo in zip([df, df_adimplente, df_inadimplente], titulos, eixos):

    df_to_plot = dataframe[coluna].value_counts().to_frame().reset_index()
    df_to_plot.columns = [coluna, 'frequencia_absoluta']

    # Manually sort the categories in alphabetical order
    categories = ['120k+', '40k - 60k', '60k - 80k', '80k - 120k', 'menos que 40k']

    # Use hue to distinguish between dataframes
    f = sns.barplot(x=coluna, y='frequencia_absoluta', data=df_to_plot, ax=eixo, hue='frequencia_absoluta', dodge=False,
                    palette=[colors[cat] for cat in categories])
    
    # Set the title and x-axis label
    f.set(title='Salarios dos clientes', xlabel='Salario Anual', ylabel='Frequência Absoluta')
    f.set_xticklabels(labels=categories, rotation=90)

    # Set the same y-axis limit for better comparison
    max_y_f = f.get_ylim()[1]
    max_y = max_y_f if max_y_f > max_y else max_y
    f.set(ylim=(0, max_y))

figura.show()


### Numerical Views

In this section, we will delve into the correlation between the response variable **default** and the numeric attributes, shedding light on the quantitative aspects of our analysis.

In [ ]:
df.drop(['id', 'default'], axis=1).select_dtypes('number').head(n=5)

- Number of Transactions in the Last 12 Months

In [ ]:
coluna = 'qtd_transacoes_12m'
titulos = ['Qtd. de Transações no Último Ano', 'Qtd. de Transações no Último Ano de Adimplentes', 'Qtd. de Transações no Último Ano de Inadimplentes']

figura, eixos = plt.subplots(1, 3, figsize=(20, 5), sharex=True)

for i, dataframe in enumerate([df, df_adimplente, df_inadimplente]):
    ax = eixos[i]

    # Use histplot for better representation of numerical data
    f = sns.histplot(x=coluna, data=dataframe, stat='count', ax=ax, bins=20)

    # Set plot labels and title
    f.set(title=titulos[i], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')

    # Set y-axis limit for consistency across subplots
    f.set(ylim=(0, max_y))

# Display the plot
figura.show()

- Value of Transactions in the Last 12 Months

In [ ]:
coluna = 'valor_transacoes_12m'
titulos = ['Valor das Transações no Último Ano', 'Valor das Transações no Último Ano de Adimplentes', 'Valor das Transações no Último Ano de Inadimplentes']

eixo = 0
max_y = 0
figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharex=True)

for dataframe in [df, df_adimplente, df_inadimplente]:

  f = sns.histplot(x=coluna, data=dataframe, stat='count', ax=eixos[eixo])
  f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')

  _, max_y_f = f.get_ylim()
  max_y = max_y_f if max_y_f > max_y else max_y
  f.set(ylim=(0, max_y))

  eixo += 1

figura.show()

- Value of Transactions in the Last 12 Months x Number of Transactions in the Last 12 Months

In [ ]:
# Define a custom color palette with distinct colors
custom_palette = {0: 'blue', 1: 'red'}

# Use sns.scatterplot with the custom color palette
f = sns.scatterplot(x='valor_transacoes_12m', y='qtd_transacoes_12m', data=df, hue='default', palette=custom_palette)

# Set plot labels and title
f.set(
    title='Relação entre Valor e Quantidade de Transações no Último Ano',
    xlabel='Valor das Transações no Último Ano',
    ylabel='Quantidade das Transações no Último Ano'
)

# Improve legend placement for better visibility
f.legend(loc='upper right')

# Display the plot
plt.show()


# 7. Summary of insights generated

- **Relationship between education and non-compliance**: There is a positive correlation between the level of education and the likelihood of being non-compliant, as shown by the higher proportion of non-compliant clients among those with doutorado e mestrado.

- **Relationship between income and compliance**: There is a positive correlation between the annual salary and the likelihood of being defaulting, as shown by the higher frequency of defaulting clients among those with highest income brackets, and the lower frequency of defaulting clients among those with lowest income brackets.

- **Income distribution of clients**: The overall client base has a skewed distribution of income, with most clients earning between $81k and $120k annually, and fewer clients earning above or below that range. The compliant clients follow a similar pattern, but with slightly lower frequencies across all income brackets.

- **Transaction frequency and compliance**: There is a positive correlation between the quantity of transactions in the last year and the likelihood of being compliant, as shown by the higher frequency of compliant clients among those with 60-80 transactions, and the higher frequency of defaulting clients among those with fewer transactions (35-45).

- **Transaction distribution of clients**: The overall client base has a normal distribution of transaction quantities, with most clients having 60-80 transactions in the last year. The compliant clients follow a similar pattern, but with a slightly higher frequency across all transaction ranges.

- **Relationship between value and quantity of transactions and clients**: The most part of transactins are located in the lower end of the value scale and also represent the focus of the mass of the higher percentage of clients.

- **Transaction types and characteristics**: There is an overlap region where both types of transactions (low value and high valur) are mixed, indicating a transitional zone where the characteristics of the transactions change.